In [ ]:
!pip install transformers datasets torch

from transformers import MBartForConditionalGeneration, MBart50TokenizerFast, Trainer, TrainingArguments
from datasets import load_dataset, Dataset
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from google.colab import drive

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/articles_100.xlsx'

Mounted at /content/drive


In [ ]:
df = pd.read_excel(file_path)
df = df[['text', 'summary']]

In [ ]:
train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [ ]:
from transformers import BartForConditionalGeneration, AutoTokenizer

# Load the saved IndicBART model and tokenizer
model_name = "ai4bharat/indicbart"
tokenizer = AutoTokenizer.from_pretrained(model_name)  # Use AutoTokenizer for compatibility
model = BartForConditionalGeneration.from_pretrained(model_name)

print("Model and tokenizer loaded successfully.")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/498 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/832 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.90M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/221 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/398 [00:00<?, ?B/s]

You are using a model of type mbart to instantiate a model of type bart. This is not supported for all configurations of models and can yield errors.


pytorch_model.bin:   0%|          | 0.00/976M [00:00<?, ?B/s]

Model and tokenizer loaded successfully.


In [ ]:
# max_length = 512

# def preprocess_data(examples):
#     # Ensure all inputs and targets are strings and replace NaN with an empty string
#     inputs = [str(text) if pd.notna(text) else "" for text in examples["text"]]
#     targets = [str(summary) if pd.notna(summary) else "" for summary in examples["summary"]]

#     # Tokenize inputs
#     model_inputs = tokenizer(inputs, max_length=max_length, truncation=True, padding="max_length")

#     # Tokenize targets
#     labels = tokenizer(targets, max_length=max_length, truncation=True, padding="max_length")

#     # Add labels to model inputs
#     model_inputs["labels"] = labels["input_ids"]
#     return model_inputs

# # Tokenize datasets
# train_dataset = train_dataset.map(preprocess_data, batched=True)
# test_dataset = test_dataset.map(preprocess_data, batched=True)
max_length = 512

def preprocess_data(examples):
    # Ensure all inputs and targets are strings and replace NaN with an empty string
    inputs = [str(text) if pd.notna(text) else "" for text in examples["text"]]
    targets = [str(summary) if pd.notna(summary) else "" for summary in examples["summary"]]

    # Tokenize inputs
    model_inputs = tokenizer(inputs, max_length=max_length, truncation=True, padding="max_length")

    # Tokenize targets using `text_target`
    labels = tokenizer(text_target=targets, max_length=max_length, truncation=True, padding="max_length")

    # Add labels to model inputs
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Tokenize datasets
train_dataset = train_dataset.map(preprocess_data, batched=True)
test_dataset = test_dataset.map(preprocess_data, batched=True)


Map:   0%|          | 0/209 [00:00<?, ? examples/s]

Map:   0%|          | 0/24 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForSeq2Seq
from torch.utils.data import DataLoader
import os

output_dir = "/content/drive/MyDrive/IndicBartresults"
os.makedirs(output_dir, exist_ok=True)

batch_size = 4  # Adjust based on GPU memory
training_args = TrainingArguments(
    output_dir=output_dir,
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=4,  # Effective batch size
    num_train_epochs=2,
    weight_decay=0.01,
    save_total_limit=1,
    save_strategy="epoch",
    bf16=False,  # Change to bf16=True if fp16 causes issues
    fp16 = False,
)

# Enable gradient checkpointing to save memory
model.gradient_checkpointing_enable()
torch.cuda.empty_cache()

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding="longest")
train_dataloader = DataLoader(train_dataset, shuffle=True, collate_fn=data_collator, batch_size=batch_size)
eval_dataloader = DataLoader(test_dataset, collate_fn=data_collator, batch_size=batch_size)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
)
trainer.train()
# # Train the model, resuming from last checkpoint if available
# if last_checkpoint:
#     trainer.train(resume_from_checkpoint=last_checkpoint)
# else:
#     trainer.train()

Epoch,Training Loss,Validation Loss
0,No log,17.818373
1,No log,17.623430


TrainOutput(global_step=26, training_loss=17.53332284780649, metrics={'train_runtime': 158.0246, 'train_samples_per_second': 2.645, 'train_steps_per_second': 0.165, 'total_flos': 223756148539392.0, 'train_loss': 17.53332284780649, 'epoch': 1.9622641509433962})

In [ ]:
import os
# Define save path and create directory if it doesn’t exist
save_path = "/content/drive/MyDrive/IndicBart_trained_5k"
os.makedirs(save_path, exist_ok=True)

# Save the model and tokenizer to the specified path
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)
print(f"Model and tokenizer saved to {save_path}")


Model and tokenizer saved to /content/drive/MyDrive/IndicBart_trained_5k


In [ ]:
def generate_summary(text):
    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True).to("cuda")
    summary_ids = model.generate(
        inputs["input_ids"],
        max_length=512,  # Shorter summaries for coherence
        num_beams=5,  # More beams for diversity
        no_repeat_ngram_size=3,  # Avoid 3-gram repetition
        length_penalty=1.2,  # Penalize overly long outputs
        early_stopping=True
    )
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# ----------------------------------------------------------------------------------
# Move model to GPU
model.to("cuda")

# List to store results
results = []

# Evaluate on the entire test set
for _, row in test_df.iterrows():
    text = row['text']
    reference_summary = row['summary']
    generated_summary = generate_summary(text)

    # Append the results as a dictionary
    results.append({
        "Original Text": text,
        "Reference Summary": reference_summary,
        "Generated Summary": generated_summary
    })

# Convert results to a DataFrame
results_df = pd.DataFrame(results)

# Save results to Excel
output_path = "/content/indicBART_test_5k.xlsx"
results_df.to_excel(output_path, index=False)

print(f"Summaries saved to {output_path}")


Summaries saved to /content/indicBART_test_5k.xlsx


In [ ]:
# from google.colab import files
# files.download("/content/summaries_20k_first_part.xlsx")
from google.colab import files
files.download("/content/indicBART_test_5k.xlsx")



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>